# Reto 01

> Consiste en encontrar la carpeta de descargas del usuario (independientemente de sus personalizaciones, tiene que funcionar si la tiene en otro lado o es un usuario cualquiera) y listar solo los archivos de la carpeta.


Sacado de https://j2logo.com/python/listar-directorio-en-python/

Para listar o recorrer un directorio:

In [2]:
import os
contenido = os.listdir('/Users/noave/Downloads')
contenido

['backupsTabGroupsFirefox',
 'borrar-cuando-termine-reto.txt',
 'desktop.ini',
 'oso-trospido.jpg',
 'Packs tipografías descargadas',
 'Telegram Desktop',
 '_editores texto y código',
 '_Libros',
 '_Portátil',
 '_Servicios Editoriales',
 '_Varios']

El primer problema es que me muestra también todas las carpetas. El segundo, que aún no sé cómo encontrar un directorio con un comodín. Al parecer, en Windows requiere acceder al registro, pero he leído que hay otro método. 

Para mostrar los archivos de tipo jpg de una carpeta:

In [3]:
import os
ejemplo_dir = '/Users/noave/Downloads'
contenido = os.listdir(ejemplo_dir)
imagenes = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ejemplo_dir, fichero)) and fichero.endswith('.jpg'):
        imagenes.append(fichero)
print(imagenes)

['oso-trospido.jpg']


Con scandir() puedo listar los ficheros de un directorio. Es la opción recomendada.

In [4]:
import os
ejemplo_dir = '/Users/noave/Downloads'
with os.scandir(ejemplo_dir) as ficheros:
    for fichero in ficheros:
        print(fichero.name)

backupsTabGroupsFirefox
borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg
Telegram Desktop
_editores texto y código
_Libros
_Portátil
_Servicios Editoriales
_Varios


Y también está iterdir() del módulo pathlib, también recomendado y que permite filtrar con is_file() o is_dir().

In [5]:
import pathlib
ejemplo_dir = '/Users/noave/Downloads'
directorio = pathlib.Path(ejemplo_dir)
for fichero in directorio.iterdir():
    print(fichero.name)

backupsTabGroupsFirefox
borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg
Telegram Desktop
_editores texto y código
_Libros
_Portátil
_Servicios Editoriales
_Varios


Para recorrer solo los ficheros de un directorio, dos métodos con lo que hemos visto. 

In [7]:
import os
ejemplo_dir = '/Users/noave/Downloads'
with os.scandir(ejemplo_dir) as ficheros:
    ficheros = [fichero.name for fichero in ficheros if fichero.is_file()]
print(ficheros)

['borrar-cuando-termine-reto.txt', 'desktop.ini', 'oso-trospido.jpg']


In [14]:
import pathlib

ejemplo_dir = '/Users/noave/Downloads/'

directorio = pathlib.Path(ejemplo_dir)
ficheros = [fichero.name for fichero in directorio.iterdir() if fichero.is_file()]
for i in ficheros:
    print(i)

borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg


In [10]:
#Y si solo queremos las carpetas:

import os
ejemplo_dir = '/Users/noave/Downloads/'
with os.scandir(ejemplo_dir) as ficheros:
    subdirectorios = [fichero.name for fichero in ficheros if fichero.is_dir()]
print(subdirectorios)

['backupsTabGroupsFirefox', 'Telegram Desktop', '_editores texto y código', '_Libros', '_Portátil', '_Servicios Editoriales', '_Varios']


Pues ya tenemos un ganador. Ahora solo nos falta descubrir la manera de que Python encuentre la carpeta de descargas, esté donde esté. 

In [18]:
DIRECTORIO_BASE_2 = os.getcwd()
print(DIRECTORIO_BASE_2)


C:\Users\noave\Python


Se me ha ocurrido que a lo mejor puedo aprovechar la información del archivo desktop.ini. A lo mejor tiene info común independientemente de la ruta y el nombre de usuario. Esto es lo que ponía:

In [19]:
[.ShellClassInfo]
LocalizedResourceName=@%SystemRoot%\system32\shell32.dll,-21798
IconResource=%SystemRoot%\system32\imageres.dll,-184

SyntaxError: invalid syntax (Temp/ipykernel_3664/3758580414.py, line 1)

https://stackoverflow.com/questions/35851281/python-finding-the-users-downloads-folder
Alguien dio esta posible solución, pero como toca registro, no me atrevo a meterla tal cual:

"""import os

def get_download_path():
    """Returns the default downloads path for linux or windows"""
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
        return location
    else:
        return os.path.join(os.path.expanduser('~'), 'downloads')"""

Este me inspira más confianza, aunque de novatos mis cojones:

"""import os

if os.name == 'nt':
    import ctypes
    from ctypes import windll, wintypes
    from uuid import UUID

    #ctypes GUID copied from MSDN sample code
    class GUID(ctypes.Structure):
        _fields_ = [
            ("Data1", wintypes.DWORD),
            ("Data2", wintypes.WORD),
            ("Data3", wintypes.WORD),
            ("Data4", wintypes.BYTE * 8)
        ] 

        def __init__(self, uuidstr):
            uuid = UUID(uuidstr)
            ctypes.Structure.__init__(self)
            self.Data1, self.Data2, self.Data3, \
                self.Data4[0], self.Data4[1], rest = uuid.fields
            for i in range(2, 8):
                self.Data4[i] = rest>>(8-i-1)*8 & 0xff

    SHGetKnownFolderPath = windll.shell32.SHGetKnownFolderPath
    SHGetKnownFolderPath.argtypes = [
        ctypes.POINTER(GUID), wintypes.DWORD,
        wintypes.HANDLE, ctypes.POINTER(ctypes.c_wchar_p)
    ]

    def _get_known_folder_path(uuidstr):
        pathptr = ctypes.c_wchar_p()
        guid = GUID(uuidstr)
        if SHGetKnownFolderPath(ctypes.byref(guid), 0, 0, ctypes.byref(pathptr)):
            raise ctypes.WinError()
        return pathptr.value

    FOLDERID_Download = '{374DE290-123F-4565-9164-39C4925E467B}'

    def get_download_folder():
        return _get_known_folder_path(FOLDERID_Download)
else:
    def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")
        
print(get_download_folder())"""
        

In [23]:
def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")
        
print(get_download_folder())

C:\Users\noave\Downloads


Vale, me ha funcionado. Supongo que si moviese la carpeta a otra parte, dejaría de funcionar (en realidad es la ruta por defecto home+downloads). Pero ya es algo. 

Ahora solo me falta… Hacer que imprima los archivos de ese directorio. 

In [25]:
def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")
    
print(get_download_folder())        

import pathlib

ejemplo_dir = get_download_folder()

directorio = pathlib.Path(ejemplo_dir)
ficheros = [fichero.name for fichero in directorio.iterdir() if fichero.is_file()]
for i in ficheros:
    print(i)

C:\Users\noave\Downloads
borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg


Reto conseguido. ¿Funcionará con otros usuarios?

In [26]:
import os

def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")
    
print(get_download_folder()) 

ejemplo_dir = '/Users/noave/Downloads'
with os.scandir(ejemplo_dir) as ficheros:
    ficheros = [fichero.name for fichero in ficheros if fichero.is_file()]
for i in ficheros:
    print(i)

C:\Users\noave\Downloads
borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg
